In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error,accuracy_score
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score
from sklearn import model_selection
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

import lightgbm as lgb
import xgboost as xgb
from xgboost import XGBClassifier

from sklearn.ensemble import RandomForestClassifier

import seaborn as sns
import matplotlib.pylab as plt
%matplotlib inline

/Users/vtrokhymenko/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
!ls -l data

total 695592
-rw-r--r--@ 1 vtrokhymenko  staff      19080 Oct 20 21:53 data_dict.xlsx
-rw-r--r--@ 1 vtrokhymenko  staff         23 Oct 22 23:15 sample_submission_ROqqAAN.csv
-rw-r--r--@ 1 vtrokhymenko  staff  137650635 Oct 22 23:24 test_plBmD8c.csv
-rw-r--r--@ 1 vtrokhymenko  staff  206939073 Oct 20 21:05 train.csv


In [179]:
pd.read_csv('data/sample_submission_ROqqAAN.csv')

,CUSTOMER_ID,RESPONDERS


In [66]:
df=pd.read_csv('data/train.csv')
df.head()

,CUSTOMER_ID,ACT_TYPE,GENDER,LEGAL_ENTITY,OCCUP_ALL_NEW,AGE,N_CASA_MAX_BALANCE_MTD,N_CASA_MIN_BALANCE_MTD,BL_TAG,TWL_TAG,...,CREDIT_ACT,DEBIT_ACT,CEL_TAG,EL_TAG,AGL_TAG,CVL_TAG,AL_TAG,HL_EMI,SCRUB_EMI,RESPONDERS
0,385793,CSA,M,INDIVIDUAL,SALARIED,21,0.00,0.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,N
1,286891,CSA,M,INDIVIDUAL,SALARIED,23,8243.77,7.77,NaN,NaN,...,Y,Y,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,N
2,108751,CSA,M,INDIVIDUAL,SALARIED,41,26515.00,0.00,NaN,NaN,...,Y,Y,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,N
3,278651,CSA,M,INDIVIDUAL,SALARIED,28,65917.28,8270.28,NaN,NaN,...,Y,Y,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,N
4,23637,CSA,M,INDIVIDUAL,SALARIED,34,22260.94,6060.94,NaN,NaN,...,Y,Y,NaN,NaN,NaN,NaN,NaN,NaN,251.758794,N


In [67]:
df.shape

(300000, 333)

In [74]:
df.RESPONDERS.value_counts()

True     295388
False      4612
Name: RESPONDERS, dtype: int64

In [90]:
df.RESPONDERS.value_counts(normalize=True)

True     0.984627
False    0.015373
Name: RESPONDERS, dtype: float64

In [69]:
d = {'N' : 1, 'Y' : 0}
df['RESPONDERS'] = df['RESPONDERS'].map(d)

In [ ]:
df['RESPONDERS'] = df['RESPONDERS'].astype(bool)

In [73]:
df.fillna(0,inplace=True)

In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Columns: 333 entries, CUSTOMER_ID to RESPONDERS
dtypes: bool(1), float64(236), int64(4), object(92)
memory usage: 760.2+ MB


---

In [172]:
test=pd.read_csv('data/test_plBmD8c.csv')
test.head()

,CUSTOMER_ID,ACT_TYPE,GENDER,LEGAL_ENTITY,OCCUP_ALL_NEW,AGE,N_CASA_MAX_BALANCE_MTD,N_CASA_MIN_BALANCE_MTD,BL_TAG,TWL_TAG,...,EOP_MON_03,CREDIT_ACT,DEBIT_ACT,CEL_TAG,EL_TAG,AGL_TAG,CVL_TAG,AL_TAG,HL_EMI,SCRUB_EMI
0,15800,CSA,M,INDIVIDUAL,SALARIED,35,1280.74,1197.74,NaN,NaN,...,17697.74,Y,Y,NaN,NaN,NaN,NaN,NaN,NaN,2557.670455
1,463196,CSA,M,INDIVIDUAL,SALARIED,27,8375.36,59.36,NaN,NaN,...,1377.99,Y,Y,NaN,NaN,NaN,NaN,NaN,NaN,1744.382609
2,309959,SA,F,INDIVIDUAL,SALARIED,37,42.00,42.00,NaN,NaN,...,42.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
3,102348,CSA,F,INDIVIDUAL,SALARIED,31,0.00,0.00,NaN,NaN,...,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
4,114850,CSA,F,INDIVIDUAL,SALARIED,28,207084.79,0.00,NaN,NaN,...,216580.99,Y,Y,NaN,NaN,NaN,NaN,NaN,NaN,2220.316265


In [16]:
test.shape

(200000, 332)

In [115]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Columns: 332 entries, CUSTOMER_ID to SCRUB_EMI
dtypes: float64(238), int64(4), object(90)
memory usage: 506.6+ MB


In [173]:
test_new=test.select_dtypes(include=['float64','int64'])
test_new.fillna(0,inplace=True)
test_new.head()

,CUSTOMER_ID,AGE,N_CASA_MAX_BALANCE_MTD,N_CASA_MIN_BALANCE_MTD,EEG_TAG,BBG_TAG,EEG_CLOSED,BBG_CLOSED,DC_SPEND_MON_03,DC_SPEND_MON_02,...,CC_SPEND_MON_03_JER,CC_SPEND_MON_03_HTL,CC_SPEND_MON_03_ATM,SAL_MON_01,SAL_MON_02,SAL_MON_03,EOP_MON_02,EOP_MON_03,HL_EMI,SCRUB_EMI
0,15800,35,1280.74,1197.74,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,20000.0,1197.74,17697.74,0.0,2557.670455
1,463196,27,8375.36,59.36,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,17095.0,12241.0,4930.0,1059.36,1377.99,0.0,1744.382609
2,309959,37,42.00,42.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,42.00,42.00,0.0,0.000000
3,102348,31,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.000000
4,114850,28,207084.79,0.00,0.0,0.0,0.0,0.0,2373.0,6918.2,...,0.0,0.0,0.0,11500.0,50000.0,20000.0,207084.79,216580.99,0.0,2220.316265


In [134]:
test_new.shape

(200000, 242)

In [174]:
test_new=test_new[list(df_new.columns.drop(['RESPONDERS']))]

---

In [129]:
test_new.select(list(df_new.columns.drop(['RESPONDERS'])), inplace=True)

TypeError: select() got an unexpected keyword argument 'inplace'

In [132]:
test_new.columns# columns(['AGE'])

Index(['BBG_TAG', 'BBG_CLOSED'], dtype='object')

## new datasen where will be ['float64','int64']

In [79]:
df_new=df.select_dtypes(include=['float64','int64','bool'])
df_new.head()

,CUSTOMER_ID,AGE,N_CASA_MAX_BALANCE_MTD,N_CASA_MIN_BALANCE_MTD,EEG_TAG,EEG_CLOSED,DC_SPEND_MON_03,DC_SPEND_MON_02,DC_SPEND_MON_01,CC_SPEND_MON_04,...,CC_SPEND_MON_03_HTL,CC_SPEND_MON_03_ATM,SAL_MON_01,SAL_MON_02,SAL_MON_03,EOP_MON_02,EOP_MON_03,HL_EMI,SCRUB_EMI,RESPONDERS
0,385793,21,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.000000,True
1,286891,23,8243.77,7.77,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,10754.0,8733.0,17154.0,3016.77,3.02,0.0,0.000000,True
2,108751,41,26515.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,9999.00,14858.75,0.0,0.000000,True
3,278651,28,65917.28,8270.28,0.0,0.0,10960.0,795.0,0.0,6782.0,...,0.0,0.0,38952.0,15200.0,19605.0,27163.10,49675.27,0.0,0.000000,True
4,23637,34,22260.94,6060.94,0.0,0.0,0.0,0.0,0.0,1199.0,...,0.0,0.0,0.0,0.0,0.0,21065.87,983.87,0.0,251.758794,True


In [80]:
df_new.shape

(300000, 241)

In [81]:
df_new.RESPONDERS.value_counts()

True     295388
False      4612
Name: RESPONDERS, dtype: int64

## XGBClassifier

In [82]:
%%time
model = XGBClassifier()
model.fit(df_new.drop(['RESPONDERS'],axis=1), df_new['RESPONDERS'])

CPU times: user 6min 28s, sys: 9.92 s, total: 6min 38s
Wall time: 2min 34s


In [146]:
sub = model.predict(test_new)

In [ ]:
# prepare and save submission
df_test['RESPONDERS'] = model.predict(df_test.values)[:, 1]
df_test.reset_index()[['CUSTOMER_ID', 'RESPONDERS']].to_csv('submissions/baseline.csv', encoding='utf-8', index=False)

In [149]:
submission = pd.read_csv("data/sample_submission_ROqqAAN.csv")

In [175]:
test_new["RESPONDERS"] = model.predict(test_new)

In [176]:
sub_my=test_new[['CUSTOMER_ID','RESPONDERS']]
sub_my.head()

,CUSTOMER_ID,RESPONDERS
0,15800,True
1,463196,True
2,309959,True
3,102348,True
4,114850,True


In [166]:
#sub_my.RESPONDERS=sub_my.RESPONDERS.map({0:'N', 1:'Y'})

In [184]:
sub_my.head()

,CUSTOMER_ID,RESPONDERS
0,15800,True
1,463196,True
2,309959,True
3,102348,True
4,114850,True


In [181]:
sub_my.to_csv('submissions/baseline.csv', encoding='utf-8', index=False)

In [183]:
pd.read_csv('submissions/baseline.csv').head()

,CUSTOMER_ID,RESPONDERS
0,15800,True
1,463196,True
2,309959,True
3,102348,True
4,114850,True
